# Single Layer Neural Network

딥러닝 알고리즘의 가장 기본이 되는 인공신경망(artificial neural network, ANN), 그 중에서도 single-layer neural network 모델을 구현한다. 
우리는 크게 세 가지 방식으로 학습하는 법을 배운다.

 1. Random Search
 2. h-step Search
 3. Gradient Descent 

In [2]:
import numpy as np

## Generate Dataset

In [5]:
# 0 ~ 1 사이에 균인할 값 100 개를 생성
x1 = np.random.uniform(low=0.0, high=1.0, size=100)

print(x1.shape)
x1[:10]

(100,)


array([ 0.11995538,  0.85299741,  0.07493303,  0.29728806,  0.57062582,
        0.86558544,  0.46639843,  0.94477476,  0.07797726,  0.15816112])

In [6]:
x2 = np.random.uniform(low=0.0, high=1.0, size=100)

print(x2.shape)
x2[:10]

(100,)


array([ 0.2212205 ,  0.88270963,  0.40675277,  0.40526446,  0.42490813,
        0.37767975,  0.00100176,  0.79276455,  0.853391  ,  0.42397203])

In [8]:
# 찾고자 하는 x1, x2의 계수 (0.3, 0.5), bias(0.1) 을 임의로 지정
# x1, x2, y 값을 이용하여 임의로 정한 계수와 바이어스 값인 0.3, 0.5, 0.1 을 찾아내 보자
y = 0.3 * x1 + 0.5 * x2 + 0.1

print(y.shape)
y[:10]

(100,)


array([ 0.24659686,  0.79725404,  0.32585629,  0.39181865,  0.48364181,
        0.54851551,  0.24042041,  0.7798147 ,  0.55008868,  0.35943435])

### First idea: Random Search

In [10]:
# loop 회수
num_epoch = 10000
# 에러
best_error = np.inf
# bewt_w1, best_w2, best_b 값을 찾아낸 loop 회수
best_epoch = None
# x1 에 대한 가중치
best_w1 = None
# x2 에 대한 가중치
best_w2 = None
# bias
best_b = None

for epoch in range(num_epoch):
    # 매 반복문 마다 random 값 생성
    w1 = np.random.uniform(low=0.0, high=1.0)
    w2 = np.random.uniform(low=0.0, high=1.0)
    b = np.random.uniform(low=0.0, high=1.0)

    # random 하게 생성된 w1, w2, b 값을 가지고 y_predict(예측값) 계산
    y_predict = x1 * w1 + x2 * w2 + b
    
    # 예측값과 실제값의 차이를 비교하여 에러 값 산출
    error = np.abs(y_predict - y).mean()
    
    # 금번 에러가 지난번 에러값 보다 작으면 best 값 모두 업데이트
    if error < best_error:
        best_error = error
        best_epoch = epoch
        best_w1 = w1
        best_w2 = w2
        best_b = b

        print("{0:4} w1 = {1:.5f}, w2 = {2:.5f}, b = {3:.5f}, error = {4:.5f}".format(best_epoch, best_w1, best_w2, best_b, best_error))

print("----" * 15)
print("{0:4} w1 = {1:.5f}, w2 = {2:.5f}, b = {3:.5f}, error = {4:.5f}".format(best_epoch, best_w1, best_w2, best_b, best_error))

   0 w1 = 0.82999, w2 = 0.50185, b = 0.07364, error = 0.22775
   8 w1 = 0.50327, w2 = 0.03789, b = 0.39384, error = 0.19886
  11 w1 = 0.23498, w2 = 0.14065, b = 0.28255, error = 0.09172
  30 w1 = 0.58439, w2 = 0.31754, b = 0.00198, error = 0.08732
  47 w1 = 0.41466, w2 = 0.31027, b = 0.15500, error = 0.05519
 165 w1 = 0.28815, w2 = 0.45759, b = 0.12090, error = 0.01107
5436 w1 = 0.27329, w2 = 0.51953, b = 0.10879, error = 0.00868
------------------------------------------------------------
5436 w1 = 0.27329, w2 = 0.51953, b = 0.10879, error = 0.00868


### Case 2 - h-step Search

In [37]:
num_epoch = 10000

# 임의로 초기 w1, w2, b 값을 구함
w1 = np.random.uniform(low=0.0, high=1.0)
w2 = np.random.uniform(low=0.0, high=1.0)
b = np.random.uniform(low=0.0, high=1.0)

h = 0.01

for epoch in range(num_epoch):
    # 최초에는 초기 w1, w2, b 값을 이용하고, 
    # 그 다음부터는 업데이트 된 w1, w2, b 값을 이용하여 예측값 계산
    y_predict = x1 * w1 + x2 * w2 + b
    # 예측값과 실제값의 차이를 구함
    current_error = np.abs(y_predict - y).mean()
    
    # 에러가 0.005 보다 작으면 멈춤
    if current_error < 0.005:
        break
    
    # w1 을 h 만큼 더한 후 예측값 다시 계산
    y_predict = x1 * (w1 + h) + x2 * w2 + b
    
    # w1 을 h 만큼 더한 후 계산한 예측값과 실제값의 에러 계산
    h_plus_error = np.abs(y_predict - y).mean()
    # 에러가 줄었으면 올바른 방향으로 w1 값을 조정했다고 보고 w1 을 w1 + h 값으로 업데이트
    if h_plus_error < current_error:
        w1 = w1 + h
    else:
        # 에러가 줄지 않았으면 찾아야 하는 값과 멀어졌다고 보고 w1 을 h 만큼 차감        
        y_predict = x1 * (w1 - h) + x2 * w2 + b
        # 다시 에러를 계산해서 현재 에러보다 작아졌으면 w1 을 w1 - h 값으로 업데이트
        h_minus_error = np.abs(y_predict - y).mean()
        if h_minus_error < current_error:
            w1 = w1 - h

    # w2 에 대해서도 w1과 동일한 방법으로 계산
    y_predict = x1 * w1 + x2 * (w2 + h) + b
    h_plus_error = np.abs(y_predict - y).mean()
    if h_plus_error < current_error:
        w2 = w2 + h
    else:
        y_predict = x1 * w1 + x2 * (w2 - h) + b
        h_minus_error = np.abs(y_predict - y).mean()
        if h_minus_error < current_error:
            w2 = w2 - h
    
    # bias 에 대해서도 w1과 동일한 방법으로 계산
    y_predict = x1 * w1 + x2 * w2 + (b + h)
    h_plus_error = np.abs(y_predict - y).mean()
    if h_plus_error < current_error:
        b = b + h
    else:
        y_predict = x1 * w1 + x2 * w2 + (b - h)
        h_minus_error = np.abs(y_predict - y).mean()
        if h_minus_error < current_error:
            b = b - h

print("{0} w1 = {1:.5f}, w2 = {2:.5f}, b = {3:.5f}, error = {4:.5f}".format(epoch, w1, w2, b, current_error))

73 w1 = 0.28701, w2 = 0.50724, b = 0.10101, error = 0.00388


### Third Idea - Gradient Descent

In [41]:
num_epoch = 100
learning_rate = 1.2

w1 = np.random.uniform(low=0.0, high=1.0)
w2 = np.random.uniform(low=0.0, high=1.0)
b = np.random.uniform(low=0.0, high=1.0)

for epoch in range (num_epoch):
    y_predict = w1 * x1 + w2 * x2 + b
    
    error = np.abs(y_predict - y).mean()
    if error < 0.001:
        break
    
    """
    # w1, w2 를 업데이트 하는 값이 달라야 함 (같으면 맴돌이...그려보면 알수있음)
    w1 = w1 - (y_predict - y).mean()
    w2 = w2 - (y_predict - y).mean()
    """            
    # ((y_predict - y) * x1).mean() 는 loss function 의 편미분 (공식 확인)
    w1 = w1 - learning_rate * ((y_predict - y) * x1).mean()
    w2 = w2 - learning_rate * ((y_predict - y) * x2).mean()
    # bias 를 절편으로 보지않고 feature 로 인식하면 됨
    # x3 = np.ones(100)
    # b=  b - learning_rate * ((y_predict - y) * x3).mean()
    b=  b - learning_rate * (y_predict - y).mean()
    
    print("{0:2} w1 = {1:.6f}, w2 = {2:.6f}, b = {3:.6f}, error = {4:.6f}".format(epoch, w1, w2, b, error))

print("----" * 10)      
print("{0:2} w1 = {1:.6f}, w2 = {2:.6f}, b = {3:.6f}, error = {4:.6f}".format(epoch, w1, w2, b, error))

 0 w1 = -0.306487, w2 = 0.363554, b = -0.321157, error = 1.043840
 1 w1 = 0.199660, w2 = 0.798900, b = 0.604664, error = 0.771517
 2 w1 = -0.128827, w2 = 0.447194, b = -0.106126, error = 0.592325
 3 w1 = 0.165268, w2 = 0.690037, b = 0.414760, error = 0.434072
 4 w1 = -0.013207, w2 = 0.488348, b = 0.010667, error = 0.336744
 5 w1 = 0.159583, w2 = 0.623100, b = 0.303020, error = 0.243627
 6 w1 = 0.064615, w2 = 0.506864, b = 0.072647, error = 0.191977
 7 w1 = 0.167662, w2 = 0.581090, b = 0.236127, error = 0.136233
 8 w1 = 0.118841, w2 = 0.513672, b = 0.104250, error = 0.109897
 9 w1 = 0.181503, w2 = 0.554134, b = 0.195147, error = 0.076405
10 w1 = 0.157902, w2 = 0.514713, b = 0.119197, error = 0.064317
11 w1 = 0.196950, w2 = 0.536443, b = 0.169290, error = 0.047458
12 w1 = 0.186896, w2 = 0.513163, b = 0.125166, error = 0.040219
13 w1 = 0.211951, w2 = 0.524582, b = 0.152386, error = 0.032969
14 w1 = 0.208973, w2 = 0.510672, b = 0.126430, error = 0.028467
15 w1 = 0.225591, w2 = 0.516478, b 

### Loss function 의 미분

Loss function : **W 값이 정답에서 멀어질 경우 오차가 급격히 커지도록**

$h(x) = wx + b$   

$L(x) = \frac{1}{2m} \sum_{i=0}^\infty (h(x)^i-y^i)^2$

Loss function 을 미분 :
- Loss function을 일단 간소화  
    $\frac{1}{2} (h(x)-y)^2 $


- Loss function을 미분  
    $\frac{dL(x)}{dW} = \frac{1}{2}2(h(x)-y) \frac{\partial}{\partial W}(h(x)-y) = (h(x)-y) \frac{\partial}{\partial W}(Wx-y) = (h(x)-y)x = \frac{1}{m} \sum_{i=0}^\infty (h(x)^i-y^i)x^i$ 


- 미분 공식 참조

    1.지수함수    
    $f(x) = x^2, f'(x) = 2x$      
    $f(x) = x^n, f'(x) = nx, \frac{df(x)}{dx} = nx, f(x) = x^2 + 3x + 5, f'(x) = 2x + 3 $
    
    2.체인룰 (복합함수)  
    $f(x) = g(h(x)), f'(x) = h'(x)g'(h(x))$
    
    3.편미분  
    $f(x,y) = x^3 + 3y + 1,  \frac{\partial (x,y)}{\partial x} = 3x, \frac{\partial (x,y)}{\partial y} = 3$
